In [ ]:
 [ ]:
# ## PCA on digits
# In this exercise we will experiment with PCA on digits.

# 1. Run PCA on the data (first 5000) http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html (use n_components = 784) and use ``plot_images`` to plot the 20 directions with largest variance. Use this PCA model for all subsequent computations. The directions can be found with the attribute ``components_``.
# 2. Take the first 20 data points from the data and project them onto the first $k$ components for $k \in \{1, 2,4,8,16, 32, 64\}$. Then plot them as images. What do you see?
#     **Hint:** To project them compute the length of the projection of each point onto the first $k$ directions and then compute for each image compute the linear combination of the first $k$ directions given by these directions (XZZ^T as in lecture).
# 3. Map all the data to 2D (the length of the projection on the first two directions) and make a scatter plot where you color with the label and see if there is some structure (use scatter with ``cmap = plt.cm.Paired``, like ``ax.scatter(x,y, c=lab, cmap=plt.cm.Paired)``)
# 4. Map all data to $32$ dimensions and run an SGD Classifier and output in-sample accuracy. How should you select target dimension in real life?
#    Use the following classifer (svm loss): ``clf = SGDClassifier(loss="hinge", alpha=0.01, max_iter=200, fit_intercept=True)``

# Discuss the results. Are they what you expect? Why? Why not.


# %matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.decomposition import PCA
from sklearn.linear_model import SGDClassifier
import torchvision.datasets as datasets

# Load full dataset
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=None)



def plot_images(dat, k=20, size=28):
    """ Plot the first k vectors as 28 x 28 images """
    x2 = dat[0:k,:].reshape(-1, size, size)
    x2 = x2.transpose(1, 0, 2)
    fig, ax = plt.subplots(figsize=(20,12))
    ax.imshow(x2.reshape(size, -1), cmap='bone')
    ax.set_yticks([])
    ax.set_xticks([])
    plt.show()

#data, labels =
data = mnist_trainset.data.numpy().reshape((60000, 28*28))
labels = mnist_trainset.targets.numpy()

# reduce size for speed
rp = np.random.permutation(len(labels))
dat = data[rp[:5000], :]
lab = labels[rp[:5000]]

components = None
## TASK 1
### YOUR CODE HERE
### END CODE
print('First 20 directions (eigenvectors X^T X)')
plot_images(components, 20)

## TASK 2
# take the first 20 data point and project them onto the first k components and plot for k in [1, 2,4,8,16, 32, 64]
img = dat[0:20, :]
print('Original images:')
plot_images(img, 20)
for k in [1, 2, 4, 8, 16, 32, 64]:
    ### YOUR CODE HERE
    ### END CODE
    
# map the data to 2D and plot the results colored by label
proj = None
## TASK 3
### YOUR CODE HERE
### END CODE
fig, ax = plt.subplots(figsize=(20, 12))
ax.scatter(proj[:,0], proj[:,1], c=lab, cmap=plt.cm.Paired)
plt.show()

## TASK 4
clf = SGDClassifier(loss="hinge", alpha=0.01, max_iter=200, fit_intercept=True)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import torch
from torch import optim
from torchvision import datasets, transforms
import torch.nn as nn
import numpy as np

def plot_images(dat, k=16):
    """ Plot the first k vectors as 28 x 28 images """
    size = 28 
    x2 = dat[0:k,:].reshape(-1, size, size)
    x2 = x2.transpose(1, 0, 2)
    fig, ax = plt.subplots(figsize=(20, 12))
    ax.imshow(x2.reshape(size, -1), cmap='bone')
    ax.set_yticks([])
    ax.set_xticks([])
    plt.show()


transform=transforms.Compose([
    transforms.ToTensor(),
    #transforms.Normalize((0.1307,), (0.3081,))
])

batch_size = 16
dataset1 = datasets.MNIST('../data', train=True, download=True, transform=transform)
dataset2 = datasets.MNIST('../data', train=False, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset2, batch_size=batch_size, shuffle=False)
# works like this 
for idx, (X, y) in enumerate(train_loader):
    x_vec = X.reshape(-1, 784)
    print('Input Images')
    plot_images(x_vec.numpy(), k=x_vec.shape[0]) # move to numpy only relevant when needing data in that format
    if idx > 5:
        break

In [ ]:
class AutoEncoder():
    
    def __init__(self):
        """ The paramters required to be set by fit method """
        self.W1 = None
        self.b1 = None
        self.W2 = None
        self.b2 = None

    def cost(self, X, W1, b1, W2, b2):
        """ Compute (Regularized) Least Squares Loss of neural net
        The clamp function may be usefull
        
          X: torch.tensor shape (n, d) - Data
          W1: torch.tensor shape (d, h) - weights
          b1: torch.tensor shape (1, h) - bias weight
          W2: torch.tensor shape (h, d) - weights
          b2: torch.tensor shape (1, d) - bias weight
        returns pytorch tensor with least squred cost
        """
   
        loss = None
        ### YOUR CODE HERE
        ### END CODE
        return loss
    
    def fit(self, data_loader, hidden_size=32, epochs=5):   
        """ GD Learning Algorithm for Ridge Regression with pytorch
        
         Args:
         data_loader: torch dataloader allows enumeration over data
         hidden_size: int
         epochs: int 
         
         sets 
        """
        def my_init(s_to, s_from):
            """ Standard way to initialize matrices in neural nets - you can ignore it """
            w = torch.zeros(s_to, s_from)
            b = torch.zeros(s_to, 1)
            nn.init.kaiming_uniform_(w, a=np.sqrt(5))
            bound = 1 / np.sqrt(s_from)
            nn.init.uniform_(b, -bound, bound)
            #assert False
            return torch.transpose(w, 1, 0), torch.transpose(b, 1, 0)        
        W1, b1 = my_init(hidden_size, 784)
        W2, b2 = my_init(784, hidden_size)
        for i, z in enumerate([W1, b1, W2, b2]):
            z.requires_grad_()

        sgd = optim.SGD(params={W1, b1, W2, b2}, lr=0.1, weight_decay=1e-4)
        #sgd = optim.AdamW(params={W1, b1, W2, b2}, lr=0.0001, weight_decay=1e-4)
        running_loss = 0.0
    
        for epoch in range(epochs):
            epoch_loss = 0
            epoch_count = 0
            running_loss = 0
            for idx, (X, y) in enumerate(data_loader):
                sgd.zero_grad()
                inputs = X.view(-1, 784) 
                loss = self.cost(inputs, W1, b1, W2, b2)
                epoch_loss += loss.item()
                epoch_count += 1
                #i = idx
                running_loss += loss.item()
                if idx % 10 == 9:   
                    print('Running loss: {2:.3f}'.format(epoch + 1, idx + 1,  epoch_loss/epoch_count), end='\r')
                    running_loss = 0.0

                loss.backward()
                sgd.step()
                #print(torch.norm(W1))
            print('Epoch: {0}, Mean Least Square loss: {1}'.format(epoch + 1, epoch_loss/epoch_count))

        self.W1 = W1.detach() #.numpy()
        self.W2 = W2.detach() #.numpy()
        self.b1 = b1.detach() #.numpy()
        self.b2 = b2.detach() #.numpy()
        

    def encode_decode(self, X):
        """ Compute the reconstructed inputs for plotting. Should be similar to cost
        
        Args:
         X: torch.tensor shape (n, d)
         
        Returns:
         decoded: torch.tensor shape (n, d) using self.W1, self.b1, self.W2, self.b2
        """

        decoded = None
        ### YOUR CODE HERE
        n=X.Shape[0]
        d=X.Shape[1]
        for i in range(n):
            for j in range(d):
                xhoed=max(0,(X[i,j]*self.w2+self.b2)*self.W1+self.W1)
                1^d*()
        ### END CODE
        return decoded
    
def simple_test(hidden_size=32, epochs=1):
    net = AutoEncoder()
    net.fit(data_loader=train_loader, hidden_size=hidden_size,  epochs=epochs)
    X_sample, y_sample =  next(iter(train_loader))
    print(X_sample.shape)
    x_vec = X_sample.view(-1, 784)
    with torch.no_grad():
        encoded_sample = net.encode_decode(x_vec).detach().numpy()
    print('Input Images')
    plot_images(x_vec.numpy(), k=x_vec.shape[0])    
    print('Reconstructed Images')
    plot_images(encoded_sample, k=x_vec.shape[0])
    
    

simple_test(32, epochs=10)